In [397]:
"""
@ Title: Assignment 3 第一題
@ Author: 商研1 鄭子萱 r07741023
@ Date: 2019/4/6
"""

'\n@ Title: Assignment 3 第一題\n@ Author: 商研1 鄭子萱 r07741023\n@ Date: 2019/4/6\n'

# Q1.1

1.fit():<br>
train出模型需要的參數(mu, cov) <br>
Prior定義為 outcome的比率 <br>
因為沒有限制為線性分類器，故此題並沒有使用share covariance <br>
        <br>
2.pred(): <br>
帶入test的每一筆資料 given mu and cov 求出likelihood 在與prior 相乘得到posterior <br>
算出每一個類別的posterior 比較出最大值即將此筆資料分到該類別

In [388]:
import pandas
import pickle
with open('phone_train.pickle', 'rb') as file:
    traindata = pickle.load(file)
with open('phone_test1.pickle', 'rb') as file:
    testdata = pickle.load(file)

In [389]:
X_train = traindata[['x','y','z']].values # convert to nd-array
y_train = traindata[['label']].values.ravel()
X_test = testdata[['x','y','z']].values
y_test = testdata[['label']].values.ravel()

In [392]:
from scipy.stats import multivariate_normal
import numpy as np

class mypgc:
    
    def __int__(self):
        pass

        
    def fit(self, x_train, y_train):
        self.x_train = x_train.copy()
        self.y_train = y_train.copy()
        
        self.K = len(set(self.y_train)) # assume classes ∈ {0, ..., K-1}
        self.y_list = list(set(self.y_train))
        self.gaussians = {}
        

        for k in range(self.K):
            Xk   = self.x_train[self.y_train == self.y_list[k]]         # get all the Xi of class k
            mu = Xk.mean(axis=0)   # compute their mean
            cov  = np.cov(Xk.T)      # compute their covariance
            prec = np.linalg.inv(cov) # compute the inverse of cov
            detcov = np.log(np.linalg.det(prec)) #compute the log of determinant of cov
            n = len(Xk) # number of observations in each label
            prior = n / len(self.x_train)

            self.gaussians[self.y_list[k]] = {
                                                "mu": mu,
                                                "cov": cov,
                                                "prec": prec,
                                                "detcov": detcov,
                                                "n": n,
                                                "prior": prior}
        
        return self.gaussians
            
            
            
    def pred(self, x_test):
        y_pred = []
        
        for j in range(x_test.shape[0]):
            prob_vect = np.zeros(self.K)

            for i in range(self.K):
                g = self.gaussians[self.y_list[i]]
                #mnormal = multivariate_normal(mean = g['mu'], cov = g['cov']) #likelihood
                prior = g['prior']
                prob_vect[i] = prior * multivariate_normal.pdf(x = x_test[j], mean = g['mu'], cov = g['cov'])
                
            tmp = np.argmax(prob_vect)
            y_hat = self.y_list[tmp]       
            y_pred.append(y_hat)
                    
        return y_pred
        

# Q1.2

In [396]:
# RUN
bc = mypgc()
bc.fit(X_train, y_train)
#y_pred = bc.pred(X_test)

{'Phoneontop': {'mu': array([ 8.13769555e-04, -9.69990841e+00, -1.00215800e-01]),
  'cov': array([[ 0.00184664,  0.00529325, -0.00601384],
         [ 0.00529325,  0.02519526, -0.02792123],
         [-0.00601384, -0.02792123,  0.0331182 ]]),
  'prec': array([[1380.20106989, -186.01874711,   93.79832935],
         [-186.01874711,  629.12383998,  496.62195539],
         [  93.79832935,  496.62195539,  465.9185559 ]]),
  'detcov': 17.04131855952812,
  'n': 26401,
  'prior': 0.15799804903738546},
 'Phoneonleft': {'mu': array([ 9.92639446,  0.09275717, -0.01933473]),
  'cov': array([[ 0.00192839, -0.00619274,  0.00063551],
         [-0.00619274,  0.03200844, -0.00317611],
         [ 0.00063551, -0.00317611,  0.00171996]]),
  'prec': array([[1369.95113577,  263.0134811 ,  -20.49603676],
         [ 263.0134811 ,   88.74589895,   66.6992188 ],
         [ -20.49603676,   66.6992188 ,  712.14869845]]),
  'detcov': 17.232141935734493,
  'n': 29522,
  'prior': 0.17667582302494958},
 'Phoneonright':

# Q1.3

In [400]:
# Accuracy
hit = 0.
for i in range(len(y_pred)):
    if y_pred[i]== y_test[i]:
        hit += 1
print("Accuracy:", hit/X_test.shape[0])

Accuracy: 1.0
